In [112]:
!pip install langchain_community langchain-text-splitters langchain-huggingface chromadb langchain-chroma langchain-perplexity sentence-transformers unstructured unstructured[pdf] nltk

In [113]:
from langchain_community.document_loaders import DirectoryLoader, UnstructuredFileLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

In [114]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [115]:
%pip install pytesseract

In [116]:
%pip install pdf2image

In [117]:
docs_path="/content"

In [118]:
vectorstore_path="content/error/vectorstore"
collection_name = "collection"

In [119]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [120]:
loader = DirectoryLoader(
    path=docs_path,
    glob="**/*",
    loader_cls=UnstructuredFileLoader,
)

In [121]:
document_loader = loader.load()

In [122]:
print(f"Loaded {len(document_loader)} documents.")
print(type(document_loader[0]))

Loaded 2 documents.
<class 'langchain_core.documents.base.Document'>


In [123]:
text_splitter = CharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

In [126]:
text_chunks = text_splitter.split_documents(document_loader)

In [127]:
len(text_chunks)

99

In [128]:
import shutil

# Remove existing vectorstore directory to avoid dimension mismatch issues
if os.path.exists(vectorstore_path):
    shutil.rmtree(vectorstore_path)

vector_store = Chroma.from_documents(
    documents=text_chunks,
    embedding=embedding,
    collection_name=collection_name,
    persist_directory=vectorstore_path
)

**Using Retrival Rag  **

In [129]:
import os

from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_perplexity import ChatPerplexity
from langchain_classic.chains import RetrievalQA
from google.colab import userdata


In [130]:
embedding  = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [131]:
vectorstore_path_new = "/content/new/vectorstore"
collection_name = "my_collection"

In [133]:
llm = ChatPerplexity(
    model="sonar",
    temperature=0.0,
    pplx_api_key=userdata.get("PPLX_API_KEY")
)

In [135]:
vector_store = Chroma(
    collection_name=collection_name,
    embedding_function=embedding,
    persist_directory=vectorstore_path_new
)

In [136]:
retriver = vector_store.as_retriever()

In [137]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriver,
    return_source_documents=True
)

In [138]:
query = "what does the document say about the Adaptive Raditation"
response = qa_chain.invoke({"query": query})
print("Answer: ", response["result"])

print("-"*80)
for source in response["source_documents"]:
    print("Source: ", source.metadata["source"])
    print("Content: ", source.page_content)
    print("-"*80)

Answer:  **Adaptive radiation** is the evolutionary process where organisms from a single ancestral species rapidly diversify into multiple new forms, each adapted to different environments, niches, or resources.[1][2][3]

All search results define it consistently as this rapid diversification, often triggered by ecological opportunities like new habitats (e.g., islands), key innovations, or reduced competition after events like mass extinctions.[2][3][4] Key characteristics include recent common ancestry, phenotype-environment correlations, trait utility for fitness, and bursts of speciation.[3][6]

Prominent examples across sources include:
- Darwin's finches on the Galápagos Islands, with beak variations for different diets (seeds, insects, nectar).[2][3][5]
- Cichlid fish in lakes like Tanganyika, showing rapid niche adaptation.[1][2]
- Mammals post-dinosaur extinction, evolving into running, flying, and swimming forms.[2]
- Australian marsupials and Hawaiian endemic species on iso